In [2]:
import sys
sys.path.append("../")
import pandas as pd
import openpyxl as oxl

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown

### Load data from the text file

In [3]:
with open('../data/raw/Sample Messages.txt') as f:
    sample_messages = f.read().strip().split('\n\n')
    #for message in sample_messages:
        #print(message)
       # print()  # blank line between messages

### Few Shot Examples

In [4]:
few_shot_examples = """
Message: BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.
Output: District: Colombo | Intent: Info | Priority: Low

Message: SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boat immediately.
Output: District: Gampaha | Intent: Rescue | Priority: High

Message: Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims reported.
Output: District: Kandy | Intent: Info | Priority: Low

Message: BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.
Output: District: Gampaha | Intent: Supply | Priority: High
"""


### Constraint of the prompt

In [5]:
constraints = """
    You are an emergency message classification system.

    - You MUST classify messages using the labeled examples provided.
    - News reports and informational updates MUST NOT be classified as Rescue.
    - Do NOT assume urgency unless explicitly stated.
    - If no district is mentioned, output None.
    - District must be extracted from any location mentioned in the message.
    - If multiple locations are mentioned, pick the main district (usually after parentheses or prepositions like 'in' or 'at').


    INTENT RULES:
    - Rescue: People in immediate danger.
    - Supply: Requests for essential supplies.
    - Info: News, warnings, reports, updates.
    - Other: Anything else.

    PRIORITY RULES:
    - High ONLY for Rescue or urgent Supply.
    - Low for Info and Other.

    OUTPUT FORMAT (STRICT):
    District: [Name or None] | Intent: [Rescue | Supply | Info | Other] | Priority: [High | Low]

    Do NOT add explanations or extra text.
"""

### Refactor the classification prompt using few_shot.v1

In [6]:
examples = few_shot_examples

model = pick_model('groq', 'general')
client = LLMClient('groq', model)


prompt_text, spec = render(
    'few_shot.v1',
    role='Emergency message Classifier',
    examples=few_shot_examples,
    constraints=constraints,
    format='District: {{district}} | Intent: {{intent}} | Priority: {{priority}}'
)

results = [] 

for m in sample_messages:
    full_prompt = f"""{prompt_text}Message: {m}"""

    messages = [{'role': 'user', 'content': full_prompt}]
    response = client.chat(messages, temperature=0.0)

    results.append({
        "Message": m,
        "Output": response["text"]
    })

    display(Markdown(f"**Input:** {m}\n\n**Output:** {response['text']}\n"))


    

**Input:** BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.

**Output:** District: Colombo | Intent: Info | Priority: Low


**Input:** SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boat immediately.

**Output:** District: Gampaha | Intent: Rescue | Priority: High


**Input:** Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims reported.

**Output:** District: Kandy | Intent: Info | Priority: Low


**Input:** Does anyone have extra dry rations for the camp in Gampaha?

**Output:** District: Gampaha | Intent: Supply | Priority: High


**Input:** News just in: Kelani river water level is at 7ft.

**Output:** District: None | Intent: Info | Priority: Low


...

**Input:** Road block at the entrance to the highway. Avoid the area.

**Output:** District: None | Intent: Info | Priority: Low


**Input:** We need blankets and clothes for the children.

**Output:** District: None | Intent: Supply | Priority: High


**Input:** SOS: Snake inside the house. We are on the table.

**Output:** District: None | Intent: Other | Priority: Low


**Input:** The army is distributing food in the affected areas.

**Output:** District: None | Intent: Supply | Priority: Low


**Input:** Is there any news about the missing fishermen?

**Output:** District: None | Intent: Info | Priority: Low


### Get the outputs into a proper structure

In [7]:
rows = []

for r in results:
    parts = r["Output"].split("|")
    rows.append({
        "Message": r["Message"],
        "District": parts[0].split(":")[1].strip(),
        "Intent": parts[1].split(":")[1].strip(),
        "Priority": parts[2].split(":")[1].strip()
    })

df = pd.DataFrame(rows)


### Save the output details to Excel file

In [8]:
df.to_excel("../data/output/classified_messages.xlsx", index=False, engine='openpyxl')